In [1]:
import pandas as pd
import numpy as np
from ctgan_adapter import CtganAdapter
from ctgan_benchmark import evaluate_ctgan, print_evaluation_results
from ctgan_utils import preprocess_data, get_tstr_results
import json

In [2]:
# Load configuration
print("# Load configuration")
with open("config.json", "r") as f:
    config = json.load(f)

# Load configuration


In [3]:
# 1. Load and prepare the dataset
print("# 1. Load and prepare the dataset")
data_raw = pd.read_csv("car.csv")
print(f"Columns in dataset: {data_raw.columns.tolist()}")
print(f"Dataset shape: {data_raw.shape}")
print(data_raw.head())

# 1. Load and prepare the dataset
Columns in dataset: ['Buying', 'Maint', 'Doors', 'Persons', 'Lug_boot', 'Safety', 'Class']
Dataset shape: (1728, 7)
  Buying  Maint Doors Persons Lug_boot Safety  Class
0  vhigh  vhigh     2       2    small    low  unacc
1  vhigh  vhigh     2       2    small    med  unacc
2  vhigh  vhigh     2       2    small   high  unacc
3  vhigh  vhigh     2       2      med    low  unacc
4  vhigh  vhigh     2       2      med    med  unacc


In [4]:
# 2. Preprocess data and detect categorical columns
print("\n# 2. Preprocess data and detect categorical columns")
data, categorical_columns = preprocess_data(data_raw)
print(f"Detected categorical columns: {categorical_columns}")

2025-03-30 16:14:56,024 - INFO - Converted Buying to category type (has 4 unique values)
2025-03-30 16:14:56,026 - INFO - Converted Maint to category type (has 4 unique values)
2025-03-30 16:14:56,029 - INFO - Converted Doors to category type (has 4 unique values)
2025-03-30 16:14:56,031 - INFO - Converted Persons to category type (has 3 unique values)
2025-03-30 16:14:56,033 - INFO - Converted Lug_boot to category type (has 3 unique values)
2025-03-30 16:14:56,034 - INFO - Converted Safety to category type (has 3 unique values)
2025-03-30 16:14:56,037 - INFO - Converted Class to category type (has 4 unique values)



# 2. Preprocess data and detect categorical columns
Detected categorical columns: ['Buying', 'Maint', 'Doors', 'Persons', 'Lug_boot', 'Safety', 'Class']


In [5]:
# 3. Define the target column for this dataset
print("\n# 3. Define the target column for this dataset")
target_column = "Class"
print(f"Target column: {target_column}")


# 3. Define the target column for this dataset
Target column: Class


In [6]:
# 4. Split the data into features and target
print("\n# 4. Split the data into features and target")
X = data.drop(columns=[target_column])
y = data[target_column]
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Target distribution:\n{y.value_counts()}")


# 4. Split the data into features and target
Features shape: (1728, 6)
Target shape: (1728,)
Target distribution:
Class
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64


In [7]:
# 5. Initialize and train CTGAN
print("\n# 5. Initialize and train CTGAN")
ctgan = CtganAdapter(**config["ctgan_params"])
print("Training CTGAN model...")
ctgan.fit(X, y)
print("Training completed")


# 5. Initialize and train CTGAN
Training CTGAN model...


Training Epochs:   0%|          | 1/300 [00:03<19:20,  3.88s/it]

Epoch 0, Loss D: 25.1800, Loss G: 0.8814


Training Epochs:  10%|█         | 31/300 [01:59<17:07,  3.82s/it]

Epoch 30, Loss D: 0.2164, Loss G: 0.0412


...

Epoch 240, Loss D: 0.1894, Loss G: -0.6769


Training Epochs:  90%|█████████ | 271/300 [14:17<01:20,  2.79s/it]

Epoch 270, Loss D: 0.0132, Loss G: -0.5083


Training Epochs: 100%|██████████| 300/300 [15:40<00:00,  3.13s/it]

Training completed


In [8]:
# 6. Generate synthetic data
print("\n# 6. Generate synthetic data")
n_samples = 1000  
print(f"Generating {n_samples} synthetic samples...")
synthetic_data = ctgan.generate(n_samples)
print(f"Generated {len(synthetic_data)} synthetic samples")
print("Synthetic data head:")
print(synthetic_data.head())


# 6. Generate synthetic data
Generating 1000 synthetic samples...
Generated 1000 synthetic samples
Synthetic data head:
  Buying  Maint  Doors Persons Lug_boot Safety  Class
0   high    med  5more    more      med    med    acc
1    low   high      2       2      big    med  unacc
2  vhigh    low      2       2      big   high  unacc
3  vhigh  vhigh  5more       2      med   high    acc
4    low  vhigh      3       2      med    low  unacc


In [9]:
# 7. Evaluate quality using TSTR and other metrics
print("\n# 7. Evaluate quality using TSTR and other metrics")
print("Running evaluation...")
evaluation_results = evaluate_ctgan(data, synthetic_data, target_column=target_column)
print_evaluation_results(evaluation_results)


2025-03-30 16:44:34,514 - WARNING - No numerical columns found for likelihood fitness evaluation
2025-03-30 16:44:34,542 - INFO - Encoded categorical target with mapping: {'acc': 0, 'good': 1, 'unacc': 2, 'vgood': 3}



# 7. Evaluate quality using TSTR and other metrics
Running evaluation...


c:\Users\kamal\OneDrive\Documents\Vilo\DEAKIN\TRIMESTER 1 2025\SIT378 TEAM PROJECT B\KatabaticGitRepo\sit378\lib\site-packages\xgboost\core.py:158: UserWarning: [16:44:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
2025-03-30 16:44:37,108 - INFO - Encoded categorical targets with mapping: {'acc': 0, 'good': 1, 'unacc': 2, 'vgood': 3}
c:\Users\kamal\OneDrive\Documents\Vilo\DEAKIN\TRIMESTER 1 2025\SIT378 TEAM PROJECT B\KatabaticGitRepo\sit378\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\kamal\OneDrive\Documents\Vilo\DEAKIN\TRIMESTER 1 2025\SIT378 TEAM PROJECT B\KatabaticGitRepo\sit378\lib\site-packages\xgboost\core.py:158: UserWarning

In [10]:
# 8. Extract and display TSTR results specifically
print("\n# 8. TSTR Performance Results")
tstr_results = get_tstr_results(evaluation_results)
if tstr_results is not None:
    print(tstr_results)


# 8. TSTR Performance Results
                    Accuracy        F1
LogisticRegression  0.785301  0.741253
RandomForest        0.743056  0.707837
MLP                 0.721644  0.695015
XGBoost             0.729745  0.704093


In [11]:
# 9. Save the synthetic data
print("\n# 9. Save synthetic data")
output_path = "bank_loan_synthetic.csv"
synthetic_data.to_csv(output_path, index=False)
print(f"Synthetic data saved to {output_path}")

print("\nTest completed successfully!")


# 9. Save synthetic data
Synthetic data saved to bank_loan_synthetic.csv

Test completed successfully!
